<a href="https://colab.research.google.com/github/sudhir108/patil/blob/master/Daily_Timings_Copy_test_Copy2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fuzzywuzzy

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import spacy
from spacy.tokens import Span
nlp = spacy.load("en_core_web_sm")
import pandas as pd
import time

import datetime
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re


In [ ]:
file_name='./Nov 4_Election snaps.xlsx'
out='./Nov 4_Election snaps_out.xlsx'
 
client_df = pd.read_excel(file_name,sheet_name='RTRS')
BBG_df = pd.read_excel(file_name,sheet_name='BBG')
#client_df=client_df[client_df.Analyst==analyst_name]

In [ ]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<:*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


client_df['RTRScleanText']=client_df['r message'].map(lambda s:preprocess(s)) 
BBG_df['BBGcleanText']=BBG_df['headline'].map(lambda s:preprocess(s)) 

In [ ]:
client_df['hour_minute_RTRS'] = pd.to_datetime(client_df['r msg dt'], format='%d/%m/%Y %H:%M:%S').sort_values()

BBG_df['hour_minute_BBG'] = pd.to_datetime(BBG_df['time'], errors='coerce')



BBG_df['hour_minute_BBG1']=BBG_df['hour_minute_BBG'].dt.hour 
client_df['hour_minute_RTRS1']=client_df['hour_minute_RTRS'].dt.hour

In [ ]:
data_after_the_matching_timings=pd.DataFrame()
BBG=[]
RTRS=[]

timex=0
less_time=0
more_time=0
BBG_time=[]
RTRS_time=[]
count=0

for a,b in zip(client_df['hour_minute_RTRS1'], client_df['RTRScleanText']): 
    
    timex=a
    
    less_time=timex-1
    more_time=timex+1
        
    for x,y in zip(BBG_df['hour_minute_BBG1'], BBG_df['BBGcleanText']):

        if x >less_time and x< more_time:
            count=count+1
            RTRS.append(b)
            BBG.append(y)
            BBG_time.append(x)
            RTRS_time.append(a)
            #print('times: ',count,a,less_time,more_time,x)
        else:
            continue
data_after_the_matching_timings['RTRS']=RTRS
data_after_the_matching_timings['BBG']=BBG

In [ ]:
master=data_after_the_matching_timings

def get_rate(master):
    
    x=master['BBG']
    y=master['RTRS']
    
    return fuzz.token_set_ratio(x, y)


res=master.apply(get_rate,axis=1)

master['res']=res



print('len >50',len(master[master['res']>50]))

print('len >60',len(master[master['res']>60]))


In [ ]:
merge00=pd.merge(master,
    BBG_df,
    how='left',
    on=None,
    left_on='BBG',
    right_on='BBGcleanText'
)



merge11=pd.merge(merge00,
    client_df,
    how='left',
    on=None,
    left_on='RTRS',
    right_on='RTRScleanText'
)

In [ ]:
# master=master[master['res']>50]
# master=master.drop_duplicates(subset ="BBG")
print('We have matchs for the ',len(master.RTRS),'alerts')

We have matchs for the  77029 alerts


In [ ]:
# merge11=merge11[merge11['res']>45]
merge11.columns

In [ ]:
final_merge=merge11

In [ ]:

# final_merge=final_merge.drop_duplicates(subset =['r message','hour_minute_RTRS'],keep= 'last')

In [ ]:
final_merge['RTRS_nums']=[re.findall(r'\d.\d*', i)  for i in final_merge['r message']] 
final_merge['BBG_nums']=[re.findall(r'\d.\d*', i)  for i in final_merge['headline']] 

In [ ]:



per=[]

for a,b in zip(final_merge['BBG_nums'],final_merge['RTRS_nums']):
    
    per.append(len(set(b).intersection(set(a)))+1/min(len(a)+1,len(b)+1))

In [ ]:
len(per)

In [ ]:
len(final_merge['RTRS_nums'].values)

In [ ]:
final_merge['per']=per

In [ ]:
final_merge=final_merge.sort_values(by=['per','res'], ascending=False)

In [ ]:
final_merge=final_merge.drop(columns=['RTRS', 'BBG',  'time','BBGcleanText', 'hour_minute_BBG1', 'r msg dt' ,'RTRScleanText', 'hour_minute_RTRS1',
        'RTRS_nums', 'BBG_nums'])

In [ ]:
# final_merge=final_merge.drop_duplicates(subset=['r message'],keep='first' )
# final_merge=final_merge.drop_duplicates(subset=['headline'],keep='first' )

In [ ]:
#final_merge.to_excel('./Downloads/maches/Oct 4-expected-Matches-Updated 2.xlsx')

In [ ]:
Result =[]
Win_Loss_Margin=[]
for i,j in zip(final_merge.hour_minute_BBG,final_merge.hour_minute_RTRS):
    if i>j:
        Win_Loss_Margin.append(i-j)
        Result.append('Win')
    elif j>i:
        Win_Loss_Margin.append(j-i)
        Result.append('Loss')
    else:
        Win_Loss_Margin.append('00:00:00')
        Result.append('Draw')
final_merge['Result']=Result
final_merge['Win Loss Margin (BBG time - RTRS time)']=Win_Loss_Margin

In [ ]:
Timings_ExtractorLogic_oct=pd.read_excel('./Timings-ExtractorLogic.xlsx')

In [ ]:
final_merge=pd.merge(final_merge,
    Timings_ExtractorLogic_oct,
    how='left',
    
    left_on='rics',
    right_on='RIC'
)



In [ ]:
final_merge.columns

In [ ]:
final_merge=final_merge.rename(columns = {'hour_minute_BBG': 'BBG time', 'hour_minute_RTRS':'RTRS Time','RIC':'Extractor logic yes/no','ratio': 'res'}, inplace = False)
final_merge=final_merge.drop(columns=['RIC_Name'])
final_merge['Extractor logic yes/no']=final_merge['Extractor logic yes/no'].fillna('False')

In [ ]:
final_merge.Result.value_counts()

#### final_merge

# New Section

In [ ]:
final_merge=final_merge.rename(columns={'headline':'BBG headline','r message':'RTRS headline'})

In [ ]:
final_merge.columns 

In [ ]:
final_merge=final_merge[['r key','BBG headline','BBG time', 'RTRS headline', 'RTRS Time','rics','Topics','pnac', 'Result',
       'Win Loss Margin (BBG time - RTRS time)', 'Extractor logic yes/no','per','res']]

In [ ]:
final_merge

In [ ]:
final_merge['cnt']=1

In [ ]:
idf=final_merge.groupby(['BBG headline'])['per'].transform(max) == final_merge['per']
final_merge2=final_merge[idf]
final_merge2

In [ ]:
maximums=final_merge2.sort_values(by=['res','per'], ascending=False)
final_merge2


In [ ]:
final_merge2=final_merge2.drop_duplicates(subset ="BBG headline",keep='first')
final_merge2=final_merge2.drop_duplicates(subset ="RTRS headline",keep='first')
final_merge2

In [ ]:
len(final_merge2[final_merge2.res>50])

End



In [ ]:
final_merge2.to_excel(out)